In [19]:
!pip install --user nltk

# !pip install lxml

## Retreive document

In [20]:
from lxml import etree

def parse_xml(xml_file):
    tree = etree.parse(xml_file)
    root = tree.getroot()
    
    documents = []
    for doc in root.findall('.//doc'):  # Look for all <doc> elements
        docno = doc.find('.//docno').text
        title = doc.find('.//title').text
        author = doc.find('.//author').text
        bib = doc.find('.//bib').text
        text = doc.find('.//text').text
        documents.append((docno, title, author, bib, text))
    return documents


documents = parse_xml("cran.all.1400.xml")
print("len(documents):", len(documents))
for row in documents:
    if row[4] is None:
        print(row)
    
documents = [row for row in documents if row[4] is not None]

print("len(documents):", len(documents))
print(documents[:5])  # Display first 5 documents for inspection


len(documents): 1400
('471', None, None, None, None)
('995', None, None, None, None)
len(documents): 1398
[('1', 'experimental investigation of the aerodynamics of a\nwing in a slipstream .', 'brenckman,m.', 'j. ae. scs. 25, 1958, 324.', 'experimental investigation of the aerodynamics of a\nwing in a slipstream .\n  an experimental study of a wing in a propeller slipstream was\nmade in order to determine the spanwise distribution of the lift\nincrease due to slipstream at different angles of attack of the wing\nand at different free stream to slipstream velocity ratios .  the\nresults were intended in part as an evaluation basis for different\ntheoretical treatments of this problem .\n  the comparative span loading curves, together with\nsupporting evidence, showed that a substantial part of the lift increment\nproduced by the slipstream was due to a /destalling/ or\nboundary-layer-control effect .  the integrated remaining lift\nincrement, after subtracting this destalling lift, was f

In [21]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### process text

In [22]:

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')  # Download WordNet data
nltk.download('stopwords')  # Download stopwords list
from nltk.tokenize import word_tokenize


def preprocess_text(text):
    # Remove newline characters and extra spaces
    text = re.sub(r'\s+', ' ', text)  # Replace all kinds of whitespace with a single space
    text = text.strip()  # Remove leading/trailing whitespace
    text = text.lower()

    text = re.sub(r'[^\w\s]', '', text)  # Keep only alphanumeric and whitespace characters
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # Apply stemming
    stemmer = PorterStemmer()
    filtered_tokens = [stemmer.stem(word) for word in filtered_tokens]
    return filtered_tokens

example_text = "The quick brown fox is running fast but it looks like he is flying for fuck sake!"
processed_text = preprocess_text(example_text)
print(processed_text)


['quick', 'brown', 'fox', 'run', 'fast', 'look', 'like', 'fli', 'fuck', 'sake']


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Create Inverted Index Matrix

In [23]:
from collections import defaultdict

def build_inverted_index(documents):
    inverted_index = defaultdict(list)  # {term: [(doc_id, freq), ...]}
    
    for doc_id, title, author, bib, text in documents:
        if not text:  # Skip documents without text
            continue
        else:
            if not author:
                author = ""
            if not bib:
                bib = ""
            processed_text = preprocess_text(title + " " + text )  # Combine title & text
            term_freq = defaultdict(int)
            
            # Count term frequency in the document
            for term in processed_text:
                term_freq[term] += 1
            
            # Add term and frequency to the inverted index
            for term, freq in term_freq.items():
                inverted_index[term].append((doc_id, freq))
    return inverted_index

inverted_index = build_inverted_index(documents)
print(dict(list(inverted_index.items())[:5]))  # Display first 5 terms in the index


{'experiment': [('1', 3), ('11', 1), ('12', 1), ('16', 1), ('17', 1), ('19', 1), ('25', 1), ('29', 1), ('30', 2), ('35', 1), ('37', 1), ('41', 1), ('43', 1), ('47', 1), ('52', 2), ('53', 1), ('58', 1), ('69', 1), ('70', 1), ('74', 2), ('78', 2), ('84', 3), ('99', 2), ('101', 1), ('103', 1), ('112', 1), ('115', 1), ('121', 1), ('123', 3), ('131', 1), ('137', 1), ('140', 1), ('142', 1), ('154', 1), ('156', 1), ('167', 1), ('168', 1), ('170', 1), ('171', 2), ('173', 2), ('176', 1), ('179', 2), ('183', 1), ('184', 1), ('186', 3), ('187', 1), ('188', 1), ('189', 2), ('191', 1), ('195', 3), ('197', 2), ('202', 1), ('203', 1), ('206', 2), ('207', 2), ('212', 1), ('216', 1), ('220', 1), ('222', 1), ('225', 2), ('227', 1), ('230', 1), ('234', 4), ('245', 1), ('251', 1), ('256', 3), ('257', 1), ('262', 1), ('271', 3), ('273', 1), ('277', 1), ('282', 1), ('283', 1), ('286', 1), ('287', 1), ('289', 1), ('294', 1), ('295', 1), ('304', 1), ('307', 1), ('329', 2), ('330', 2), ('334', 2), ('338', 1), 

In [24]:
import math

def compute_tf(freq, doc_length, max_freq = 0, method="augmented"):
    if method == "raw":
        return freq / doc_length
    elif method == "log":
        return 1 + math.log(freq) if freq > 0 else 0
    elif method == "augmented":
        return 0.5 + (0.5 * freq / max_freq)
    return 0


In [25]:
print(documents[:2])

[('1', 'experimental investigation of the aerodynamics of a\nwing in a slipstream .', 'brenckman,m.', 'j. ae. scs. 25, 1958, 324.', 'experimental investigation of the aerodynamics of a\nwing in a slipstream .\n  an experimental study of a wing in a propeller slipstream was\nmade in order to determine the spanwise distribution of the lift\nincrease due to slipstream at different angles of attack of the wing\nand at different free stream to slipstream velocity ratios .  the\nresults were intended in part as an evaluation basis for different\ntheoretical treatments of this problem .\n  the comparative span loading curves, together with\nsupporting evidence, showed that a substantial part of the lift increment\nproduced by the slipstream was due to a /destalling/ or\nboundary-layer-control effect .  the integrated remaining lift\nincrement, after subtracting this destalling lift, was found to agree\nwell with a potential flow theory .\n  an empirical evaluation of the destalling effects wa

# Compute TF-IDF matric for VSM

In [26]:
import math
from collections import defaultdict

def compute_tf_idf(inverted_index, documents):
    """Compute the TF-IDF matrix from an inverted index."""
    N = len(documents)  # Total number of documents
    tf_idf = defaultdict(dict)  # Store TF-IDF scores

    # Compute document lengths based on preprocessed text
    doc_lengths = {doc[0]: len(preprocess_text(doc[4])) for doc in documents}
    avg_doc_length = sum(doc_lengths.values()) / N  # Compute average document length

    print("Sample doc length:", doc_lengths['1'])  # Verify document length
    print("Summed TF from index:", sum(inverted_index[term]['1'] for term in inverted_index if '1' in inverted_index[term]))

    for term, doc_freqs in inverted_index.items():
        df = len(doc_freqs)  # Number of documents containing the term
        if df == 0:
            print(f"Warning: Term '{term}' has df=0, check indexing!")

        idf = math.log((N + 1) / (df + 1)) + 1  # Smoothed IDF to avoid zero division

        for (doc_id, freq) in doc_freqs:
            doc_length = doc_lengths[doc_id]  # Get the length of the document
            tf = (1 + math.log(freq)) if freq > 0 else 0  # Logarithmic TF
            
            # BM25-like length normalization
            tf /= (1 - 0.75 + 0.75 * (doc_length / avg_doc_length))

            tf_idf_score = tf * idf  # Compute final TF-IDF score

            tf_idf[doc_id][term] = tf_idf_score  # Store the score

    return tf_idf

tf_idf = compute_tf_idf(inverted_index, documents)
print(dict(list(tf_idf.items())[:5]))  # Display first 5 documents in the TF-IDF matrix


Sample doc length: 77
Summed TF from index: 0
{'1': {'experiment': 5.737841054382719, 'investig': 4.498056615657707, 'aerodynam': 5.88820625210848, 'wing': 7.718907909927602, 'slipstream': 17.67374383820254, 'studi': 3.1256046045635446, 'propel': 5.50807897937778, 'made': 4.546207163792112, 'order': 3.6442894977632214, 'determin': 2.868829620446816, 'spanwis': 5.547717444047467, 'distribut': 2.6628827758605262, 'lift': 8.843078186426647, 'increas': 3.287975856727465, 'due': 6.261100412521749, 'differ': 7.078507506621823, 'angl': 3.3212012029332643, 'attack': 4.022866775442388, 'free': 4.043981041249094, 'stream': 3.622067528674192, 'veloc': 2.8281529384874706, 'ratio': 2.9110259821991606, 'result': 1.9230912840291214, 'intend': 6.414393773893241, 'part': 6.939466588013481, 'evalu': 7.613525437098513, 'basi': 4.528946690092039, 'theoret': 3.1398731774225555, 'treatment': 5.08971203276697, 'problem': 2.7818959420778144, 'compar': 3.083849448202598, 'span': 5.4697844533932525, 'load': 3.3

In [27]:
print(preprocess_text("experimental investigation of the aerodynamics of a wing in a slipstream."))


['experiment', 'investig', 'aerodynam', 'wing', 'slipstream']


In [28]:
# Create a lookup dictionary for document titles and full texts
doc_lookup = {doc[0]: (doc[1], doc[4]) for doc in documents}  # { "1": ("Title 1", "Full text..."), ... }

def display_results(ranked_results, doc_lookup, top_n=10):
    """Display the top N ranked results with their titles and allow the user to read one."""
    print("\nTop Search Results:\n")
    for rank, (doc_id, score) in enumerate(ranked_results[:top_n], start=1):
        title = doc_lookup.get(doc_id, ("Unknown Title", ""))[0]
        print(f"{rank}. [{doc_id}] {title} (Score: {score:.4f})")

    # Ask user if they want to read a document
    doc_id_to_read = input("\nEnter a document ID to read (or press Enter to skip): ").strip()
    if doc_id_to_read in doc_lookup:
        title, text = doc_lookup[doc_id_to_read]
        print(f"\n=== {title} ===\n{text[:500]}...")  # Show only first 500 characters
    else:
        print("No document selected or invalid ID.")

# display_results(ranked_results, doc_lookup)

In [29]:
import xml.etree.ElementTree as ET

def parse_queries(filename):
    """Parse queries from the Cranfield XML file."""
    queries = {}
    tree = ET.parse(filename)
    root = tree.getroot()
    
    for top in root.findall("top"):
        num = top.find("num").text.strip()  # Extract query number
        title = top.find("title").text.strip()  # Extract query text
        
        if num and title:
            queries[num] = title

    return queries


In [30]:
def min_max_normalize(scores):
    """Normalize scores using Min-Max scaling, keeping document IDs as strings."""
    scores = {str(doc_id): float(score) for doc_id, score in scores.items()}  # Ensure float values
    min_score = min(scores.values())
    max_score = max(scores.values())
    if max_score - min_score == 0:
        return {doc_id: 0.0 for doc_id in scores}  # Avoid division by zero
    return {doc_id: (score - min_score) / (max_score - min_score) for doc_id, score in scores.items()}


In [31]:
import numpy as np
def compute_cosine_similarity(tf_idf, query, inverted_index, N):
    """Compute cosine similarity between query and documents."""
    query_tf_idf = {}

    # Compute TF-IDF for query using the same IDF values as the document matrix
    for term in query:
        if term in inverted_index:
            df = len(inverted_index[term])  # Document frequency
            idf = math.log(N / df) if df > 0 else 0  # Compute IDF
            tf = compute_tf(query.count(term), len(query), method="raw")  # Query term frequency
            query_tf_idf[term] = tf * idf  # TF-IDF for query

    # Compute cosine similarity for each document
    scores = {}
    for doc_id, doc_vector in tf_idf.items():
        doc_norm = np.linalg.norm(list(doc_vector.values()))  # Document vector norm
        query_norm = np.linalg.norm(list(query_tf_idf.values()))  # Query vector norm

        # Compute dot product
        dot_product = sum(doc_vector.get(term, 0) * query_tf_idf.get(term, 0) for term in query_tf_idf)

        # Compute cosine similarity
        if doc_norm > 0 and query_norm > 0:
            scores[doc_id] = dot_product / (doc_norm * query_norm)
        else:
            scores[doc_id] = 0

    # Normalize scores
    if type(scores) == str:
        print(scores)
    # scores = min_max_normalize(scores)
    
    # Rank documents by score
    ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:100]
    return ranked_docs

In [32]:
vsm_results = {}

queries = parse_queries("cran_updated.qry.xml")  # Load queries from XML file
print("len(queries):", len(queries))
# print(queries)

for query_id, query_text in queries.items():
    query_text = preprocess_text(query_text)  # Apply the same preprocessing as indexing
    ranked_docs = compute_cosine_similarity(tf_idf, query_text, inverted_index, len(documents))
    vsm_results[query_id] = ranked_docs
    # if query_id == '1':
    #     print("query = ", query_text)
    #     display_results(ranked_docs, doc_lookup, top_n=10)


len(queries): 225


In [33]:
def save_results_trec_format(ranked_results, model_name, output_file):
    """
    Save ranked results in TREC format: query_id Q0 doc_id rank score model_name
    """
    with open(output_file, 'w') as f:
        for query_id, results in ranked_results.items():
            for rank, (doc_id, score) in enumerate(results, start=1):
                f.write(f"{query_id} 0 {doc_id} {rank} {score:.4f} {model_name}\n")

# Save VSM results
save_results_trec_format(vsm_results, "VSM", "vsm_results_just_text.txt")


# BM25

In [ ]:
import math
from collections import defaultdict

def compute_bm25_2(inverted_index, documents, queries, k1=1.5, b=0.75):
    N = len(documents)  # Total number of documents
    doc_lengths = {doc[0]: len(doc[4].split()) for doc in documents}  # Document lengths
    avg_doc_length = sum(doc_lengths.values()) / N  # Average document length
    
    # Compute IDF for each term
    idf = {}
    for term, doc_freqs in inverted_index.items():
        df = len(doc_freqs)  # Number of documents containing the term
        idf[term] = math.log((N - df + 0.5) / (df + 0.5) + 1)
    
    bm25_scores = defaultdict(dict)  # Store BM25 scores
    
    for query_id, query_text in queries.items():
        query_terms = preprocess_text(query_text)
        
        for term in query_terms:
            if term in inverted_index:
                for doc_id, tf in inverted_index[term]:
                    doc_length = doc_lengths[doc_id]
                    tf_weight = (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (doc_length / avg_doc_length)))        # compute tf weights
                    bm25_scores[query_id][doc_id] = bm25_scores[query_id].get(doc_id, 0) + idf[term] * tf_weight
    
    return bm25_scores


In [16]:
import math
from collections import defaultdict

def compute_bm25(inverted_index, documents, queries, k1=1.5, b=0.75):
    """
    Compute BM25 scores for documents given a query.
    
    Args:
        inverted_index (dict): Inverted index mapping terms to (doc_id, tf) pairs.
        documents (list): List of documents, where each document is a tuple (doc_id, title, author, bib, text).
        queries (dict): Dictionary of queries, where keys are query IDs and values are query texts.
        k1 (float): Controls term frequency saturation. Default is 1.5.
        b (float): Controls document length normalization. Default is 0.75.
    
    Returns:
        dict: BM25 scores for each query, mapped to document IDs.
    """
    N = len(documents)  # Total number of documents
    doc_lengths = {doc[0]: len(doc[4].split()) for doc in documents}  # Document lengths
    avg_doc_length = sum(doc_lengths.values()) / N  # Average document length
    
    # Compute IDF for each term using Robertson-Sparck Jones formula
    idf = {}
    for term, doc_freqs in inverted_index.items():
        df = len(doc_freqs)  # Number of documents containing the term
        idf[term] = math.log((N - df + 0.5) / (df + 0.5) + 1)
    
    bm25_scores = defaultdict(dict)  # Store BM25 scores
    
    for query_id, query_text in queries.items():
        query_terms = preprocess_text(query_text)  # Preprocess query text
        
        for term in query_terms:
            if term in inverted_index:
                for doc_id, tf in inverted_index[term]:
                    doc_length = doc_lengths[doc_id]
                    
                    # Compute TF weight with saturation and document length normalization
                    tf_weight = (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (doc_length / avg_doc_length)))
                    
                    # Compute BM25 score contribution for this term
                    bm25_scores[query_id][doc_id] = bm25_scores[query_id].get(doc_id, 0) + idf[term] * tf_weight
    
    return bm25_scores

In [17]:
bm25_scores = compute_bm25(inverted_index, documents, queries)

# Format results in TREC format and save
with open("bm25_results2.txt", "w") as f:
    for query_id, scores in bm25_scores.items():
        ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:1000]  # Keep top 1000 results
        for rank, (doc_id, score) in enumerate(ranked_docs, start=1):
            f.write(f"{query_id} 0 {doc_id} {rank} {score:.4f} BM25\n")

print("BM25 results saved.")

BM25 results saved.


# N GRAM

# RESTART

In [141]:
def generate_ngrams(text, n):
    return [' '.join(text[i:i+n]) for i in range(len(text) - n + 1)]


from collections import defaultdict, Counter

def build_ngram_index(documents, n):
    """Build an inverted index for n-grams with frequencies."""
    ngram_index = defaultdict(dict)  # Now stores {ngram: {doc_id: frequency}}
    for doc_id, title, author, bib, text in documents:
        if not text:  # Skip documents without text
            continue
        else:
            if not author:
                author = ""
            if not bib:
                bib = ""
            tokens = preprocess_text(title + " " + text + " "  + author + " " + bib) # tokenises words and applies filters
            ngrams = generate_ngrams(tokens, n)
            ngram_freq = Counter(ngrams)  # Count frequency of each n-gram in the document
            for ngram, freq in ngram_freq.items():
                if doc_id in ngram_index[ngram]:
                    ngram_index[ngram][doc_id] += freq
                else:
                    ngram_index[ngram][doc_id] = freq
    print(f"Number of n-grams in index: {len(ngram_index)} and first is {list(ngram_index.keys())[0]}")
    return ngram_index


import math

def compute_ngram_similarity(ngram_index, query, documents, n):
    """Compute n-gram overlap similarity between query and documents with weighted scores."""
    try:
        query = preprocess_text(query)
        query_ngrams = generate_ngrams(query, n)
        # print(f"Query N-grams: {query_ngrams}")

        # Calculate IDF for each n-gram in the query
        N = len(documents)  # Total number of documents
        idf = {}
        for ngram in query_ngrams:
            if ngram in ngram_index:
                df = len(ngram_index[ngram])  # Document frequency of the n-gram
                idf[ngram] = math.log(N / df)
            else:
                idf[ngram] = 0  # If n-gram is not in the index, IDF is 0

        scores = defaultdict(float)
        for ngram in query_ngrams:
            if ngram in ngram_index:
                for doc_id, freq in ngram_index[ngram].items():
                    # Weighted score: frequency of n-gram in document * IDF of n-gram
                    scores[doc_id] += freq * idf[ngram]

        # print(f"Scores before normalization: {scores}")

        # Rank documents by score (limit to top 100)
        if not scores:
            return []  # Return an empty list if no scores
        else:
            ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:100]
            return ranked_docs

    except Exception as e:
        print(f"Error processing query {query}\nquery_ngrams = {query_ngrams}: {e}")

In [142]:
Number_n = 1
ngram_index = build_ngram_index(documents, n=Number_n)  # Change n as needed
# ranked_results = compute_ngram_similarity(ngram_index, queries, documents, n=2)


Number of n-grams in index: 8971 and first is experiment


In [143]:
print(ngram_index[5])

{}


In [144]:
from collections import defaultdict, Counter

def compute_collection_probabilities(documents, n):
    """Compute the probability of each n-gram in the corpus."""
    ngram_counter = Counter()
    total_ngrams = 0

    for doc_id, title, author, bib, text in documents:
        if not text:
            continue
        else :
            if not author:
                author = ""
            if not bib:
                bib = ""
            if not title:
                title = ""
            tokens = preprocess_text(title + " " + text + " " + author + " " + bib)
            ngrams = generate_ngrams(tokens, n)
            ngram_counter.update(ngrams)
            total_ngrams += len(ngrams)

    collection_prob = {ngram: count / total_ngrams for ngram, count in ngram_counter.items()}
    return collection_prob

In [145]:
all_results = {}

queries = parse_queries("cran_updated.qry.xml")  # Load queries from XML file
for query_id, query in queries.items():
    ranked_results = compute_ngram_similarity(ngram_index, query, documents, n=Number_n)
    all_results[query_id] = ranked_results

In [146]:
def save_results_trec_format2(ranked_results, model_name, output_file):
    """
    Save ranked results in TREC format: query_id Q0 doc_id rank score model_name
    """
    with open(output_file, 'w') as f:
        for query_id, results in ranked_results.items():
            for rank, (doc_id, score) in enumerate(results[:100], start=1):
                f.write(f"{query_id} 0 {doc_id} {rank} {score:.4f} {model_name}\n")

In [147]:
save_results_trec_format2(all_results, "ngram_weighted", "ngram_weighted_results2.txt")